In [3]:
import json
import psycopg2
import pandas as pd
import numpy as np

In [2]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [ ]:

def load_course(day,db,location):
  if day>=12:
    day = 11
    location = []
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  print("location : ",location)
  location_count = len(location)
  if location_count == 4 or location_count == 0:
    location_sql = ' and 1=1'
  #case 2 : 한 지역만 선택한 경우 
  elif location_count == 1:
    if day>=10:
      location_sql = ' and 1=1'
    else:
      location_sql = f" and LOWER(main_location) = \'{location[0]}\'"
  #case 3 : 여러 지역을 선택한 경우
  else:
    if location_count == 2:
      location_sql = f" and main_location = 'Mixed' and {location[0]} >0 and {location[1]} >0"
    elif location_count == 3:
      location_sql = f" and main_location = 'Mixed' and {location[0]} >0 and {location[1]} >0 and {location[2]} >0"

  df = pd.DataFrame()

  #places가 80번(러브랜드 휴업중)을 포함하는 코스는 제외
  exclude_love_land_sql = " and 80 != ALL(places)"

  load_course_sql= (f"SELECT id, theme, score, places FROM course_ WHERE length>={min_length} and length<={max_length}" + location_sql + exclude_love_land_sql)
  #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
  print("load_course_sql : " ,load_course_sql)
  cursor.execute(load_course_sql)
  result = cursor.fetchall()

  df =pd.DataFrame(result)
  
  #해당 조건에 만족하는 코스가 없는 경우
  if df.empty:
    print("no course")
    return df

  df.columns = ['id','theme','score','places']

  return df

##코스에 children pet 정보 반영하기

###데이터 불러와서 스코어 점수 벡터로 변환하기

In [4]:
children_pet = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.17_children_pet_info.csv",index_col=0)
children_pet.head(2)

,title,children,pet
id,,,
1,사려니숲길,1,0
2,우도(해양도립공원),1,1


In [6]:
children_pet.reset_index(inplace=True)

In [7]:
children_pet.head(3)

,id,title,children,pet
0,1,사려니숲길,1,0
1,2,우도(해양도립공원),1,1
2,3,성산일출봉(UNESCO 세계자연유산),0,0


In [17]:
entire_chilren_info = []
entire_pet_info = []
for i in children_pet.values:
  children_score = i[2]
  pet_score = i[3]
  
  children_vector = [0]*3
  pet_vector = [0] * 2
  
  children_vector[children_score] += 1
  pet_vector[pet_score ] += 1

  entire_chilren_info.append(children_vector)
  entire_pet_info.append(pet_vector)

In [23]:
children_pet['children_vector'] = entire_chilren_info

In [25]:
children_pet['pet_vector'] = entire_pet_info

In [26]:
children_pet = children_pet[['id','title','children_vector','pet_vector']]
children_pet

,id,title,children_vector,pet_vector
0,1,사려니숲길,"[0, 1, 0]","[1, 0]"
1,2,우도(해양도립공원),"[0, 1, 0]","[0, 1]"
2,3,성산일출봉(UNESCO 세계자연유산),"[1, 0, 0]","[1, 0]"
3,4,비자림,"[0, 1, 0]","[1, 0]"
4,5,천지연폭포,"[0, 1, 0]","[1, 0]"
...,...,...,...,...
1084,1085,헌마공신김만일기념관,"[0, 1, 0]","[0, 1]"
1085,1086,훈데르트바서파크,"[0, 1, 0]","[1, 0]"
1086,1087,세화소품관,"[0, 1, 0]","[1, 0]"
1087,1088,무비랜드왁스뮤지엄,"[0, 0, 1]","[1, 0]"


###코스 로드 후 반영

In [27]:
course = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/08.21_course.csv",index_col=0)
course.head(2)

,id,places,theme,cluster,score
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",10,167322
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",7,163116


In [28]:
course['places'] = course['places'].apply(lambda x: list(map(int,x[1:-1].split(','))))

In [29]:
course.head(2)

,id,places,theme,cluster,score
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",10,167322
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",7,163116


In [48]:
from tqdm import tqdm

In [67]:
entire_children_vec = []
entire_pet_vec = []
for now in tqdm(course.values):
  places = now[1]
  now_children_vec = np.array([0,0,0])
  now_pet_vec = np.array([0,0])
  for place in places:
    place_vec = children_pet.loc[children_pet['id'] == place,['children_vector','pet_vector','title']].values[0]
    now_children_vec += np.array(place_vec[0]) # children_vec
    now_pet_vec += np.array(place_vec[1]) # pet_vec

  #평균으로 나눠주기
  now_children_vec = now_children_vec / len(places)
  now_pet_vec = now_pet_vec / len(places)

  entire_children_vec.append(now_children_vec)
  entire_pet_vec.append(now_pet_vec)    

100%|██████████| 3725/3725 [00:32<00:00, 114.88it/s]


In [98]:
course[['children0','children1','children2']] = entire_children_vec
course[['pet0','pet1']] = entire_pet_vec

In [100]:
course = course[['id','places','theme','children0','children1','children2','pet0','pet1','score','cluster']]
course.head(5)

,id,places,theme,children0,children1,children2,pet0,pet1,score,cluster
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",0.000000,0.750000,0.250000,0.500000,0.500000,167322,10
1,2,"[58, 37, 10, 19, 602, 60, 29]","[0.14285714285714285, 0.14285714285714285, 0.5...",0.142857,0.714286,0.142857,0.428571,0.571429,163116,7
2,3,"[58, 10, 19, 602, 29]","[0.2, 0.2, 0.6, 0.0, 0.2, 0.0, 0.2, 0.6, 0.8, ...",0.000000,0.800000,0.200000,0.400000,0.600000,178099,6
3,141,"[1, 7, 8, 48, 22, 215]","[0.16666666666666666, 0.0, 0.5, 0.333333333333...",0.333333,0.500000,0.166667,0.666667,0.333333,424938,5
4,156,"[1, 8, 174, 60, 184, 28, 223]","[0.0, 0.0, 0.5714285714285714, 0.4285714285714...",0.142857,0.857143,0.000000,0.285714,0.714286,302233,2


##분포 확인

In [136]:
children_pet[['children0','children1','children2']] = entire_chilren_info

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [137]:
children_pet[['pet0','pet1']] = entire_pet_info

In [139]:
children_pet = children_pet[['id','title','children0','children1','children2','pet0','pet1']]

In [140]:
children_pet

,id,title,children0,children1,children2,pet0,pet1
0,1,사려니숲길,0,1,0,1,0
1,2,우도(해양도립공원),0,1,0,0,1
2,3,성산일출봉(UNESCO 세계자연유산),1,0,0,1,0
3,4,비자림,0,1,0,1,0
4,5,천지연폭포,0,1,0,1,0
...,...,...,...,...,...,...,...
1084,1085,헌마공신김만일기념관,0,1,0,0,1
1085,1086,훈데르트바서파크,0,1,0,1,0
1086,1087,세화소품관,0,1,0,1,0
1087,1088,무비랜드왁스뮤지엄,0,0,1,1,0


In [141]:
children_pet[children_pet['children0']==1]

,id,title,children0,children1,children2,pet0,pet1
2,3,성산일출봉(UNESCO 세계자연유산),1,0,0,1,0
6,7,새별오름,1,0,0,1,0
10,11,거문오름(UNESCO 세계자연유산),1,0,0,1,0
13,14,한라산국립공원,1,0,0,1,0
20,21,산굼부리,1,0,0,0,1
...,...,...,...,...,...,...,...
1061,1062,솔담한방병원,1,0,0,1,0
1062,1063,고포잇다이브센터,1,0,0,1,0
1063,1064,그제바람 윈드서핑패들보드,1,0,0,1,0
1069,1070,케이제주해양사업단,1,0,0,1,0


###기준 세우기1 - children


In [127]:
course[(course['children0']==0)].shape #아이가 싫어하는 곳이 있는 코스

(1185, 10)

In [107]:
course[course['children2']>0].shape #아이만을 위한 곳이 하나 이상 있는 코스

(1089, 10)

In [133]:
course.loc[(course['children0']==0) & (course['children2']>0) ] #아이가 싫어하는 곳이 없고, 아이만을 위한 여행지가 있는 코스

,id,places,theme,children0,children1,children2,pet0,pet1,score,cluster
0,1,"[602, 10, 58, 29]","[0.25, 0.25, 0.5, 0.0, 0.25, 0.0, 0.25, 0.5, 0...",0.0,0.750000,0.250000,0.500000,0.500000,167322,10
2,3,"[58, 10, 19, 602, 29]","[0.2, 0.2, 0.6, 0.0, 0.2, 0.0, 0.2, 0.6, 0.8, ...",0.0,0.800000,0.200000,0.400000,0.600000,178099,6
6,4,"[202, 58, 37]","[0.0, 0.3333333333333333, 0.3333333333333333, ...",0.0,0.666667,0.333333,0.333333,0.666667,96780,11
10,3229,"[1, 2, 9, 47, 319]","[0.4, 0.0, 0.2, 0.4, 0.2, 0.0, 0.4, 0.4, 0.4, ...",0.0,0.800000,0.200000,0.400000,0.600000,603079,5
14,3233,"[233, 10, 22]","[0.3333333333333333, 0.0, 0.6666666666666666, ...",0.0,0.666667,0.333333,0.333333,0.666667,207657,10
...,...,...,...,...,...,...,...,...,...,...
3691,3689,"[224, 1, 2, 1058, 78, 15, 112, 22, 153, 30]","[0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2, 0.4, 0.6, ...",0.0,0.800000,0.200000,0.600000,0.400000,334690,10
3694,3692,"[4, 5, 6, 41, 78, 15, 18, 1075, 22, 88, 28, 93...","[0.07692307692307693, 0.0, 0.5384615384615384,...",0.0,0.846154,0.153846,0.538462,0.461538,266371,5
3698,3696,"[4, 38, 230, 8, 6, 103, 78, 15, 49, 17, 22, 86...","[0.14285714285714285, 0.07142857142857142, 0.4...",0.0,0.928571,0.071429,0.428571,0.571429,254031,7
3716,3716,"[1, 2, 5, 6, 37, 8, 78, 15, 18, 274, 84, 22, 2...","[0.0625, 0.0625, 0.5625, 0.3125, 0.0625, 0.062...",0.0,0.937500,0.062500,0.375000,0.625000,357075,5


###기준세우기2 - pet

In [109]:
course[course['pet1'] == 1].shape #모든 여행지가 반려동물 가능한 곳

(152, 10)

In [113]:
course[course['pet1'] >= 0.5].shape #5할의 여행지가 반려동물 가능한 곳

(2675, 10)

In [115]:
course[course['pet1'] >= 0.7].shape #7할의 여행지가 반려동물 가능한 곳

(976, 10)

####아이에게 어울리지 않는 곳의 비율

In [142]:
course.describe()

,id,children0,children1,children2,pet0,pet1,score,cluster
count,3725.000000,3725.000000,3725.000000,3725.000000,3725.000000,3725.000000,3725.000000,3725.000000
mean,1863.000000,0.134981,0.813957,0.051063,0.425710,0.574290,284033.856644,5.610470
std,1075.459204,0.129914,0.147594,0.098393,0.199264,0.199264,136608.418795,3.390365
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12340.000000,0.000000
25%,932.000000,0.000000,0.733333,0.000000,0.300000,0.444444,180998.000000,3.000000
50%,1863.000000,0.125000,0.833333,0.000000,0.416667,0.583333,272125.000000,6.000000
75%,2794.000000,0.200000,0.916667,0.076923,0.555556,0.700000,368976.000000,8.000000
max,3725.000000,1.000000,1.000000,0.666667,1.000000,1.000000,953742.000000,11.000000


In [155]:
course.loc[(course['children0']<=0.125) & (course['children2']>0) ].shape #아이가 싫어하는 곳이 없고, 아이만을 위한 여행지가 있는 코스

(694, 10)

###DB에 반영하기

In [120]:
db = load_db()
cursor = db.cursor()

In [122]:
db.rollback()

In [123]:
cnt = 0
for i in tqdm(course.values):
  id = i[0]
  children0,children1,children2,pet0,pet1 = i[3:8]
  #print(id,children0,children1,children2,pet0,pet1)
  sql = f'update course_ set children0={children0}, children1 = {children1}, children2 = {children2}, pet0 = {pet0}, pet1={pet1} where id = {id}'
  cursor.execute(sql)
db.commit()

100%|██████████| 3725/3725 [08:42<00:00,  7.13it/s]


##load_course 함수 수정

In [177]:

def load_course(day,db,location,children,pet):
  if day>=12:
    day = 11
    location = []
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  print("location : ",location)
  location_count = len(location)
  location_sql = ''
  if location_count == 4 or location_count == 0:
    location_sql = ' and 1=1'
  #case 2 : 한 지역만 선택한 경우 
  elif location_count == 1:
    if day>=10:
      location_sql = ' and 1=1'
    else:
      location_sql = f" and LOWER(main_location) = \'{location[0]}\'"
  #case 3 : 여러 지역을 선택한 경우
  else:
    if location_count == 2:
      location_sql = f" and main_location = 'Mixed' and {location[0]} >0 and {location[1]} >0"
    elif location_count == 3:
      location_sql = f" and main_location = 'Mixed' and {location[0]} >0 and {location[1]} >0 and {location[2]} >0"

  df = pd.DataFrame()

  #places가 80번(러브랜드 휴업중)을 포함하는 코스는 제외
  exclude_love_land_sql = " and 80 != ALL(places)"

  load_course_sql= (f"SELECT id, theme, score, places, children0,children2, pet1 FROM course_ WHERE length>={min_length} and length<={max_length}" + location_sql + exclude_love_land_sql)
  #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
  print("load_course_sql : " ,load_course_sql)
  cursor.execute(load_course_sql)
  result = cursor.fetchall()

  df =pd.DataFrame(result)
  
  #해당 조건에 만족하는 코스가 없는 경우
  if df.empty:
    print("no course")
    return df
#(course['children0']<0.125) & (course['children2']>0) ].
  df.columns = ['id','theme','score','places','children0','children2','pet1']
  if children == 1:
    df = df.loc[ (df['children0']<=0.125) & (df['children2']>0) ]
  if pet == 1:
    df = df.loc[ df['pet1'] >= 0.5]
  return df

In [ ]:
#def load_course(day,db,location,children,pet):

In [182]:
day= 4
location='east,west,south,north'
children_info=1
pet_info=0
test_df = load_course(day=5,db=db,location=location,children=children_info,pet=1)

location :  east,west,south,north
load_course_sql :  SELECT id, theme, score, places, children0,children2, pet1 FROM course_ WHERE length>=10 and length<=20 and 80 != ALL(places)


In [183]:
test_df

,id,theme,score,places,children0,children2,pet1
6,3254,"[0.0, 0.1, 0.9, 0.0, 0.0, 0.1, 0.2, 0.7, 0.7, ...",278151,"[1, 200, 8, 41, 10, 174, 22, 119, 91, 60]",0.100000,0.100000,0.700000
9,52,"[0.08333333333333333, 0.0, 0.5833333333333334,...",276788,"[2, 134, 8, 10, 13, 19, 25, 22, 310, 153, 94, 31]",0.000000,0.083333,0.916667
12,64,"[0.3333333333333333, 0.08333333333333333, 0.33...",189410,"[66, 4, 295, 10, 15, 145, 83, 24, 153, 61, 184...",0.083333,0.166667,0.500000
33,145,"[0.07692307692307693, 0.07692307692307693, 0.5...",229656,"[2, 200, 8, 41, 174, 15, 18, 83, 52, 24, 185, ...",0.076923,0.076923,0.615385
37,150,"[0.18181818181818182, 0.0, 0.18181818181818182...",150169,"[194, 37, 5, 7, 103, 300, 589, 79, 244, 121, 90]",0.090909,0.090909,0.636364
...,...,...,...,...,...,...,...
1196,3668,"[0.3125, 0.25, 0.25, 0.1875, 0.25, 0.125, 0.06...",106486,"[66, 58, 38, 103, 78, 15, 16, 114, 83, 630, 21...",0.062500,0.312500,0.500000
1210,3686,"[0.11764705882352941, 0.0, 0.5294117647058824,...",235996,"[3, 5, 71, 8, 9, 202, 59, 14, 15, 49, 84, 244,...",0.117647,0.176471,0.529412
1217,3696,"[0.14285714285714285, 0.07142857142857142, 0.4...",254031,"[4, 38, 230, 8, 6, 103, 78, 15, 49, 17, 22, 86...",0.000000,0.071429,0.571429
1223,3704,"[0.058823529411764705, 0.0, 0.4705882352941176...",405132,"[2, 3, 4, 37, 5, 6, 8, 9, 76, 13, 15, 112, 49,...",0.058824,0.058824,0.588235


In [176]:
course.loc[(course['children0']<=0.125) & (course['children2']>0) & (course['pet1']>0.5) ].shape #아이가 싫어하는 곳이 없고, 아이만을 위한 여행지가 있는 코스

(304, 10)